In [1]:
import sys
from bs4 import BeautifulSoup
import requests
import datetime as dt
from PIL import Image
import helpers as hf

import yaml
from unidecode import unidecode
import pandas as pd
from PIL import Image, ImageOps, ImageDraw, ImageFont
import json
import numpy as np


with open("info/design_config.yml", "r") as f:
    design_config = yaml.safe_load(f)

design_config,shop_config = hf.get_config('cbb')
cfbd_loader = hf.cfbp_handler(design_config['cfbd_api'])

private file size: 2466


In [2]:
ify_user = hf.shopify_printify(shop_config,'cbb')

In [3]:
# check if printify ptoduct is posted
headers_printify = {
    "Authorization": f"Bearer {shop_config['printify_access']}",
    "Content-Type": "application/json",
}
headers_shopify = {
            "X-Shopify-Access-Token": shop_config["shopify_access"],
            "Content-Type": "application/json",
        }

In [9]:
prods2 = ify_user.recur_get_products(products=[])

In [10]:
print(len(prods))
print(len(prods2))

472
236


In [ ]:
import time

In [ ]:
def recur_get_products(self, products=[]):
    products_link = f"https://{self.post_dict[self.version]['shop_name']}.myshopify.com/admin/api/2024-01/products.json?limit=250"
    response = requests.get(products_link, headers=self.headers_shopify)
    products.extend(response.json()["products"])
    if "next" in response.links.keys():
        # theres another page of products
        if response.headers["X-RateLimit-Remaining"] == 0:
            time.sleep(1)
        return self.recur_get_products(
            response.links["next"]["url"], products=products
        )
    else:
        return products

In [ ]:
def recursive_get_prods(self, since_id=0, all_products=[]):
    url = f"https://{self.post_dict[self.version]['shop_name']}.myshopify.com/admin/api/2023-01/products.json?since_id={since_id}"
    response = requests.get(url=url, headers=self.headers_shopify)
    products = json.loads(response.text)["products"]
    if len(products) == 0:
        return all_products
    else:
        # new products were found, there could be more in the next iteration
        all_products.extend(products)
        return self.recursive_get_prods(
            since_id=products[-1]["id"], all_products=all_products
        )


In [7]:
len(ids)

236

In [6]:
ids = []
for p in prods:
    id = p['id']
    if id not in ids:
        ids.append(id)
    if p["images"][0]["alt"] is not None:
        print(p)

{'id': 8368514007261, 'title': 'Washington Vs California. 17 Feb, 2024', 'body_html': 'Unisex! Sizes are in mens.\n AI image representing the basketball game 80 to 82 between Washington and California, played at  17 Feb, 2024 \nWashington Huskies basketball shirt California Golden Bears basketball shirt\n||Washington|| ||California||ROUND: ||None||', 'vendor': 'Printify', 'product_type': 'T-Shirt', 'created_at': '2024-02-19T19:26:14-08:00', 'handle': 'washington-vs-california-17-feb-2024', 'updated_at': '2024-02-20T17:49:07-08:00', 'published_at': '2024-02-19T19:26:16-08:00', 'template_suffix': None, 'published_scope': 'web', 'tags': 'basketball, CaliforniaCalifornia, college, madness, march, marchmadness, sports, Washington, Washingtonbasketball', 'status': 'active', 'admin_graphql_api_id': 'gid://shopify/Product/8368514007261', 'variants': [{'id': 44574512808157, 'product_id': 8368514007261, 'title': 'Sport Grey / S', 'price': '25.99', 'sku': '21748886559083817971', 'position': 1, 'i